<a href="https://colab.research.google.com/github/minipri/Fiap_BigData/blob/main/Desafio_Titanc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
# Carga Bibliotecas e Dados
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

train = pd.read_csv("https://raw.githubusercontent.com/diogenesjusto/FIAP/master/SHIFT/Data/titanic/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/diogenesjusto/FIAP/master/SHIFT/Data/titanic/test.csv")
     


In [116]:
# Tratar valores ausentes - Age
train['Age'].fillna(train['Age'].median(), inplace=True)

# Tratar valores ausentes - Embarked 
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True) 

# Tirar colunas que não importam
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [112]:
# Preparação/Modelo
mod = LinearRegression()
X = train.loc[:,['Sex','Pclass','SibSp','Parch','Embarked']]
# Variável Embarked é tratada como Dummy (caso similar as meses do Pib), visto que é uma variável categórica
X = pd.get_dummies(X)
# DICA DE PYTHON (muitos perguntam): Se você desejar UNIR dois dataframes utilize o comando=> X = pd.concat([X, X], axis=1) <= no exemplo "dentro" dos sinais > e <, o dataframe X é "duplicado" lado a lado. Faça o teste
Y = train.loc[:,['Survived']]
mod.fit(X ,Y )

X_test = test.loc[:,['Sex','Pclass','SibSp','Parch','Embarked']]
X_test = pd.get_dummies(X_test)
p = mod.predict(X_test)

# As linhas abaixo só montam o arquivo de saída que será utilizado no Kaggle, com destaque para
# como o resultado da variável p é um número entre 0 e 1, convertemos ele para 0 se for menor que 0.5 e para 1 se maior
Survived_Predict = [1 if  i > 0.5 else 0 for i in p]
df_pred = test.loc[:,['PassengerId']]
df_pred['Survived'] = Survived_Predict

#from google.colab import files
#df_pred.to_csv('predictSexPclassSibSpParchEmb.csv', index=False) 
#files.download('predictSexPclassSibSpParchEmb.csv')

 #Esse modelo por enquanto é o melhor, com 0.77272


In [73]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    int64  
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(1), int64(5), object(2)
memory usage: 55.8+ KB


In [63]:
train
train.to_csv('trainalterada.csv', index=False) 
files.download('trainalterada.csv') #criei essa para ver como estava a base depois de alterada

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
train['Age'] = train['Age'].astype(int) #alterei formato para int para ver se resolvia o problema do NaN, mas não funcionou

In [66]:
column_stats = train['Age'].describe() #criei essa para entender melhor a composição da base, mas estou achando estranho o dtype que aparece
print(column_stats)

count    891.000000
mean      29.345679
std       13.028212
min        0.000000
25%       22.000000
50%       28.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64


In [117]:



# Preparação/Modelo
mod = LinearRegression()
X = train.loc[:,['Sex', 'Pclass','SibSp','Parch','Age']]

# Variável Embarked é tratada como Dummy (caso similar as meses do Pib), visto que é uma variável categórica
X = pd.get_dummies(X)

imputer = SimpleImputer(strategy='mean') #criei pq estava dando um erro da NaN e eu não encontrava onde era
X = imputer.fit_transform(X)

# DICA DE PYTHON (muitos perguntam): Se você desejar UNIR dois dataframes utilize o comando=> X = pd.concat([X, X], axis=1) <= no exemplo "dentro" dos sinais > e <, o dataframe X é "duplicado" lado a lado. Faça o teste

Y = train.loc[:,['Survived']]
mod.fit(X ,Y )

X_test = test.loc[:,['Sex', 'Pclass','SibSp','Parch','Age']]
X_test = pd.get_dummies(X_test)
X_test = imputer.fit_transform(X_test) #criei pq estava dando um erro da NaN e eu não encontrava onde era

p = mod.predict(X_test)

# As linhas abaixo só montam o arquivo de saída que será utilizado no Kaggle, com destaque para
# como o resultado da variável p é um número entre 0 e 1, convertemos ele para 0 se for menor que 0.5 e para 1 se maior
Survived_Predict = [1 if  i > 0.5 else 0 for i in p]
df_pred = test.loc[:,['PassengerId']]
df_pred['Survived'] = Survived_Predict

#from google.colab import files
#df_pred.to_csv('predictSexPclassSibSpParchAge.csv', index=False) 
#files.download('predictSexPclassSibSpParchAge.csv')
     
 #Esse modelo por enquanto não é o melhor, com 0.77073

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [127]:
# Criar a variável FamilySize somando as colunas SibSp e Parch
train['FamSize'] = train['SibSp'] + train['Parch']

# Criar uma variável IsAlone indicando se a pessoa estava sozinha ou acompanhada
train['IsAlone'] = 0
train.loc[train['FamSize'] == 0, 'IsAlone'] = 1

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,IsAlone,FamSize
0,0,3,male,22.0,1,0,7.2500,S,0,1
1,1,1,female,38.0,1,0,71.2833,C,0,1
2,1,3,female,26.0,0,0,7.9250,S,1,0
3,1,1,female,35.0,1,0,53.1000,S,0,1
4,0,3,male,35.0,0,0,8.0500,S,1,0


In [131]:
# Criar a variável FamilySize somando as colunas SibSp e Parch
test['FamSize'] = test['SibSp'] + test['Parch']

# Criar uma variável IsAlone indicando se a pessoa estava sozinha ou acompanhada
test['IsAlone'] = 0
test.loc[train['FamSize'] == 0, 'IsAlone'] = 1

test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamSize,IsAlone
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,2,1


In [192]:
# Preparação/Modelo
mod = LinearRegression()
X = train.loc[:,['Sex', 'Pclass','Age','Embarked', 'FamSize']]

# Variável Embarked é tratada como Dummy (caso similar as meses do Pib), visto que é uma variável categórica
X = pd.get_dummies(X)

imputer = SimpleImputer(strategy='mean') #criei pq estava dando um erro da NaN e eu não encontrava onde era
X = imputer.fit_transform(X)

# DICA DE PYTHON (muitos perguntam): Se você desejar UNIR dois dataframes utilize o comando=> X = pd.concat([X, X], axis=1) <= no exemplo "dentro" dos sinais > e <, o dataframe X é "duplicado" lado a lado. Faça o teste

Y = train.loc[:,['Survived']]
mod.fit(X ,Y )

X_test = test.loc[:,['Sex', 'Pclass','Age','Embarked', 'FamSize']]
X_test = pd.get_dummies(X_test)
X_test = imputer.fit_transform(X_test) #criei pq estava dando um erro da NaN e eu não encontrava onde era

p = mod.predict(X_test)

# As linhas abaixo só montam o arquivo de saída que será utilizado no Kaggle, com destaque para
# como o resultado da variável p é um número entre 0 e 1, convertemos ele para 0 se for menor que 0.5 e para 1 se maior
Survived_Predict = [1 if  i > 0.5 else 0 for i in p]
df_pred = test.loc[:,['PassengerId']]
df_pred['Survived'] = Survived_Predict

#from google.colab import files
#df_pred.to_csv('predictSexPclassAgeEmbFam.csv', index=False) 
#files.download('predictSexPclassAgeEmbFam.csv')
     
 #Esse modelo por enquanto não é o melhor, com 0.77073